In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [1]:
from pathlib import Path

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
from PIL import Image

from feature_vector_generation.models import Img2Vec
from feature_vector_generation.predict import Predict

In [2]:
model = Img2Vec()
img = Image.open('kitten_small.jpg')
imgarr = np.asarray(img)

model.model_name, model.layer_output_size

Loading RESNET-18 model


('resnet-18', 512)

In [3]:
activation = {}
def getActivation(name):
    # the hook signature
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [4]:
imgs_t = [model.normalize(model.to_tensor(model.scaler(Image.open(img_pth)))) for img_pth in Path('.').glob('*.jpeg')]
imgs_np = np.array([model.normalize(model.to_tensor(model.scaler(Image.open(img_pth)))).numpy() for img_pth in Path('.').glob('*.jpeg')])

In [5]:
imgs_np = np.array([np.asarray(model.scaler(Image.open(img_pth))) for img_pth in Path('.').glob('*.jpeg')])

In [6]:
imgs_np1 = np.expand_dims(np.asarray(img), 0)
imgs_np1.shape

(1, 168, 224, 3)

In [7]:
imgs_t = torch.stack([model.normalize(model.scaler(transforms.ToTensor()(img))) for img in imgs_np])
imgs_t.shape

torch.Size([3, 3, 224, 224])

## Loading a pre-trained model

### Model 1 - resent 18

In [6]:
resnet = models.resnet18()
ext_layer = model._modules.get('avgpool')

h = ext_layer.register_forward_hook(getActivation('fvec'))

preds = resnet(imgs_t)
h.remove()

activation['fvec'].squeeze().shape

AttributeError: 'Img2Vec' object has no attribute '_modules'

### Model 2 - resnet 18

In [9]:
model = Img2Vec(model='resnet-18')

h = model.extraction_layer.register_forward_hook(getActivation('fvec'))

preds = model.model(imgs_t)
h.remove()

activation['fvec'].squeeze().shape

Loading RESNET-18 model


torch.Size([3, 512])

### Model 3 - vgg

In [11]:
vgg = Img2Vec(model='vgg')
h = vgg.extraction_layer.register_forward_hook(getActivation('fvec'))

preds = vgg.model(imgs_t)
h.remove()

activation['fvec'].squeeze().shape

Loading VGG-11 model


torch.Size([3, 4096])

### Model 4 - Alexnet

In [15]:
alexnet = Img2Vec(model='alexnet')
h = alexnet.extraction_layer.register_forward_hook(getActivation('fvec'))

preds = alexnet.model(imgs_t)
h.remove()

activation['fvec'].shape

Loading ALEXNET model


torch.Size([3, 4096])

### Loading densenet model

In [14]:
densenet = Img2Vec(model='densenet')
h = densenet.extraction_layer.register_forward_hook(getActivation('fvec'))

preds = densenet.model(imgs_t)
h.remove()

activation['fvec'].squeeze().shape

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home/srivatsas/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

torch.Size([3, 1024, 7, 7])

## Using Predict module

In [9]:
vgg = Predict(model='vgg')

vgg.get_vec_np(imgs_np).shape

Loading VGG-11 model


(3, 4096)

In [10]:
resnet = Predict()

resnet.get_vec_np(imgs_np).shape

Loading RESNET-18 model


(3, 512)

In [12]:
alexnet = Predict(model='alexnet')

alexnet.get_vec_np(imgs_np).shape

Loading ALEXNET model


(3, 4096)

In [13]:
def f():
    return 2,3

type(f())

tuple